In [7]:
import os
import sys
import pydicom
import numpy as np
import nibabel as nib

print('Python: {}'.format(sys.version))
print('Pydicom: {}'.format(pydicom.__version__))
print('Numpy: {}'.format(np.__version__))
print('Nibabel: {}'.format(nib.__version__))

Python: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Pydicom: 2.2.1
Numpy: 1.19.5
Nibabel: 3.2.1


In [8]:
############# PARAMETERS ###################
INPUT = './cteph_gz_files/'                             # input path to nii files
INPUT_DCM = './CTEPH CineCT Scans/CTEPH CineCT Scans/'  # input path to the corresponding dicom files
OUT_PATH = './ct_volumes_test/'                         # path to save generated dicom files

In [9]:
def create_dicoms(image, dcm, out_path, starting_pos=-100.0):
    
    """
    Creates dicom files corresponding to each slice of a nii file.
    
    Inputs: 
        image - nii image file (opened, image data extracted)
        dcm - corresponding dicom header (dicom header opened with pydicom)
        starting_pos - location in space to start first slice (float)
        
    Returns:
        None 
    """
    
    # extract necessary info from dicom headers
    patientID = str(dcm[0x0008, 0x0090].value)
    
    series = dcm.SeriesNumber
    old_id = dcm.SOPInstanceUID
    studyID = dcm[0x0020, 0x0010].value

    pos = starting_pos
    spacing = dcm.SliceThickness
    dcm.add_new([0x0028,0x0008], 'US', image.shape[3])

    # iterate over each position in the nii volume
    for i in range(image.shape[2]):

        # find new filename, update instance number
        patient_pos = dcm[0x0020, 0x0032].value
        patient_pos[2] = pos
        dcm[0x0020, 0x0032].value = patient_pos

        dcm.SliceLocation = pos
        dcm[0x0027, 0x1050].value = pos
        dcm[0x0027, 0x1051].value = pos
        dcm[0x0027, 0x1044].value = pos

        # decrease position by spacing amount
        pos -= spacing

        # get image slice from nii
        data_new = image[:,:,i,:]
        out_array = np.moveaxis(data_new, 2, 0)
        
        # save a dicom for each time point
        for j in range(out_array.shape[0]):
            
            dcm.InstanceNumber = i*10 + j
            dcm.AcquisitionNumber = j
            
            new_id = old_id + '.{}.{}'.format(i,j)
            dcm.SOPInstanceUID = new_id

            # copy the data back to the original dcm
            out = out_array[j,:,:]
            dcm.PixelData = out.tobytes()

            # update the information regarding the shape of the data array
            dcm.Rows, dcm.Columns, _ = data_new.shape

            if not os.path.isdir(out_path + patientID):
                os.mkdir(out_path + patientID)
            
            out_filename = '{}_{}_{}.dcm'.format(series, i, j)
            dcm.save_as(out_path + '/' + patientID + '/' + out_filename)

In [10]:
def volumes_from_images(image_list):
    
    """ Converts a list of nii images to a single volume """
    
    image_list = [np.expand_dims(x, axis=-1) for x in image_list]
    return np.concatenate(image_list, axis=-1)

In [11]:
# iterate through directory, storing all filepaths
nii_list = []
subdir_list = []
for (root, subdir, files) in os.walk(INPUT):
    for sdir in subdir:
        subdir_list.append(sdir)
    for file in files:
        nii_list.append(os.path.join(root, file))
        
# select just one patient for this analysis
subdir_list = [subdir_list[2]]        # comment out this line to run on all patients
subdir_list

['CVC1901261322']

In [12]:
# iterate over each patient
for patient in subdir_list:
    
    print('Saving dicoms for {}'.format(patient))
    
    # select nii files for that patient
    nii_files = [x for x in nii_list if patient in x]
    
    # load dcm corresponding to each patient (will only be used as a template)
    # Load relevant dicom header information
    PATH_DCM = INPUT_DCM + '\{}'.format(patient)

    for (root, subdir, files) in os.walk(PATH_DCM):
        for file in files[-1:]:
            dcm = pydicom.dcmread(os.path.join(root, file))
            
            if dcm.file_meta.TransferSyntaxUID.is_compressed is True:
                print('compressed data!')
                print(dcm.file_meta.TransferSyntaxUID)
                dcm.decompress()
   
    image_list = []    
    # iterate over nii files
    for file in nii_files:
        dcm.SeriesDescription = "dcm from nii file (nii2dcm.ipynb)"
            
        # load nii file and extract image info
        img = nib.load(file)
        image = np.array(img.get_fdata()).astype(np.int16)
        image_list.append(image)
        
    vol = volumes_from_images(image_list)
    
    # create dicom files
    create_dicoms(vol, dcm, OUT_PATH, -100.0)
    
    print('Done!')

Done!
